In [18]:
from __future__ import print_function
from __future__ import division
import datetime as dt
import hdbscan
import logging
import matplotlib.pyplot as plt
import matplotlib.animation as mpl_animation
from IPython.display import HTML
from fleet_clustering import bq
from fleet_clustering import filters
from fleet_clustering import distances
from fleet_clustering import animation

In [13]:
ais_by_date = bq.load_ais_by_date('drifting_longlines', dt.date(2017, 1, 1), dt.date(2017, 12, 31))

In [14]:
pruned_by_date = {k : filters.remove_chinese_coast(v) for (k, v) in ais_by_date.items()}

In [15]:
valid_ssvid = sorted(filters.find_valid_ssvid(pruned_by_date))
C = distances.create_composite_lonlat_array(pruned_by_date, valid_ssvid)

In [30]:
dists = distances.compute_distances(C, clip=1000)
clusterer = hdbscan.HDBSCAN(metric='precomputed', min_cluster_size=12)
clusterer.fit(dists)

HDBSCAN(algorithm='best', allow_single_cluster=False, alpha=1.0,
    approx_min_span_tree=True, cluster_selection_method='eom',
    core_dist_n_jobs=4, gen_min_span_tree=False, leaf_size=40,
    match_reference_implementation=False, memory=Memory(cachedir=None),
    metric='precomputed', min_cluster_size=12, min_samples=None, p=None,
    prediction_data=False)

In [31]:
anim = animation.make_anim(valid_ssvid, clusterer.labels_, pruned_by_date, interval=5)
HTML(anim.to_html5_video())

In [32]:
anim = animation.make_anim(valid_ssvid, clusterer.labels_, pruned_by_date, interval=1)
Writer = mpl_animation.writers['ffmpeg']
writer = Writer(fps=10, metadata=dict(artist='Me'), bitrate=1800)
anim.save('fleet_longliners.mp4', writer=writer)